Note to self: I should defintely explore an object-oriented PyTorch approach in the future. It is annoying having to pass around every single required variable to helper functions

Import Statements

In [55]:
import os
import torch
import torchvision
from torch import nn 
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from torch.utils.data import Dataset
from mpl_toolkits.mplot3d import Axes3D
import scipy.io

from torch.autograd.functional import jacobian
from torch.autograd.functional import hessian

Some Variables

In [56]:
#number of intermediate steps
q = 500

#step size
h = 0.8

#start and end location
start_t = 0.1
end_t = start_t + h

N = 100 #number of x values for training

Import Data

In [57]:
#CREDIT TO https://github.com/maziarraissi/PINNs/blob/master/appendix/discrete_time_inference%20(Burgers)/Burgers_systematic.py FOR THE BELOW CODE TO IMPORT IRK WEIGHTS AND THE LABEL DATA (AND FOR THE DATA ITSELF)

tmp = np.float32(np.loadtxt('./data/Butcher_IRK%d.txt' % (q), ndmin = 2))
IRK_weights = np.reshape(tmp[0:q**2+q], (q+1,q)) #presume the first q rows and q columns represent a_{ij} values, and the final row represents the b_j values
IRK_times = tmp[q**2+q:] #not sure what this is used for


#I may choose to generate my own data once more...
'''
data = scipy.io.loadmat('./data/burgers_shock.mat')

t_points = data['t'].flatten()[:,None] # T x 1
x_points = data['x'].flatten()[:,None] # N x 1
Exact = np.real(data['usol']).T # T x N

#get data labels at t = 0.9 
index = np.where(t_points == end_t)[0][0]
end_labels = Exact[index, :]'''

"\ndata = scipy.io.loadmat('./data/burgers_shock.mat')\n\nt_points = data['t'].flatten()[:,None] # T x 1\nx_points = data['x'].flatten()[:,None] # N x 1\nExact = np.real(data['usol']).T # T x N\n\n#get data labels at t = 0.9 \nindex = np.where(t_points == end_t)[0][0]\nend_labels = Exact[index, :]"

Generate Data

In [58]:
input = np.random.rand(N) * 2 - 1 #x values between -1 and 1

initial_vals = -np.sin(input * np.pi) #initial values at t = 0

Define Network

In [59]:
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.act_func = nn.Tanh()
        self.predict = nn.Sequential(
            nn.Linear(1, 50),
            self.act_func,
            nn.Linear(50, 50),
            self.act_func,
            nn.Linear(50, 50),
            self.act_func,
            nn.Linear(50, q + 1)
        )

    def forward(self, x):
        x = self.predict(x)
        return x

Helper Methods

In [60]:
#Nonelinear Burgers' Operator
def NLO(input, outputs, device): #outputs is of the form N x q. Inputs is of form N x 1 and was used to get outputs


    u = outputs
    print(input.shape)
    print(u.shape)

    u_x = jacobian(lambda u,x: u, (u, input), create_graph=True, vectorize=True)

    print(len(u_x))
    print(u_x[0].shape)
    print(u_x[1].shape)

    u_x = u_x.squeeze(-1) #Jacobian with vectorize should return shape (N, q, 1)

    u_xx = jacobian(lambda u_x, x: u_x, (u_x, input), create_graph=True, vectorize=True)

    u_xx = u_xx.squeeze(-1)


    return u * u_x - (0.01/torch.pi)*u_xx
    
    


#Implicit Runge-Kutta Calculation (Combine Intermediate Terms with Butcher Tableau Coefficients)
def IRK(input, outputs, initial_vals, coefficients, device): #outputs is tensor of size Nx(q+1), represnting q stages for N x-values and the final n+1 prediction. coefficients is a tensor of size (q+1) x q representing the Butcher Tableau. Input and initial_vals are size (N)

    outputs_truncated = outputs[:, :-1] #get all but last row

    global N
    global q
    global h


    NLO_outputs = NLO(input, outputs_truncated, device) #perform non-linear operation on each of the outputs_truncated. Shape is still (N, q)
    
    N_coefficients = coefficients.repeat(N, 1, 1) #stack N coefficients tensors on top of each other
    NLO_outputs = NLO_outputs.unsqueeze(-1) #reshape NLO_outputs_reshaped into shape (N, q, 1) for matrix multiplication

    coefficients_applied = torch.bmm(N_coefficients, NLO_outputs) #perform matrix multiplication. This represents applying the coefficients a_{ij} (and b_i for n+1 prediction) from the Butcher Tableau to every element in NLO_outputs and THEN recombining them to make every new value which will go on to form the "calculated" outputs (intermediate stage values). Has shape (N, q+1, 1), since N_coefficients has shape (N, q+1, q) and NLO_outputs has shape (N, q, 1). For each layer in N, the matrix mulitplcaion (q+1, q) x (q, 1) takes place, resulting in a vector of shape (q+1, 1). N of these makes (N, q+1, 1)
    coefficients_applied = coefficients_applied.view(N, -1) #now shape (N, q + 1)

    coefficients_applied = -h * coefficients_applied #apply the -h (-\delta t) to each element in coefficients_applied
    initial_vals = initial_vals.view(-1, 1) #reshape initial_vals from (N) to (N, 1)

    result = coefficients_applied + initial_vals #Every element in coefficients_applied has a value added to it which is the element from initial_vals on the same layer. We can add the shapes (N, q+1) and (N, 1) due to "broadcasting", which effectively stretches the shape of the vector

    return result #returns a result which is the same shape as "outputs" and represents the same intermediate values, except these are the "calculated" versions and not the "direct predictions"



Loss Functions

In [61]:
def SSE_n(input, outputs, initial_vals, coefficients, device):

    calculated_outputs = IRK(input, outputs, initial_vals, coefficients, device)

    SSE_n_loss = torch.sum((calculated_outputs - outputs)**2)

    return SSE_n_loss


def SSE_b(network, device):
    boundary_points = torch.tensor([-1, 1]).to(device).view(-1, 1)
    boundary_results = network(boundary_points) #get all intermediary points and the n+1 prediction for x = -1 and 1
    return torch.sum(boundary_results**2) #square results and sum them


def SSE(input, outputs, initial_vals, coefficients, network, device):
    SSE_n_loss = SSE_n(input, outputs, initial_vals, coefficients, device)
    SSE_b_loss = SSE_b(network, device)
    return SSE_n_loss + SSE_b_loss, SSE_n, SSE_b

Training Setup

In [62]:
pinn = PINN()
optimizer = torch.optim.LBFGS(pinn.parameters(), #PARAMETERS CREDIT TO https://github.com/teeratornk/PINNs-2/blob/master/Burgers%20Equation/Burgers%20Inference%20(PyTorch).ipynb
                              lr=1,
                              max_iter=50000, 
                                max_eval=50000, 
                                history_size=50,
                                tolerance_grad=1e-5, 
                                tolerance_change=1.0 * np.finfo(float).eps,
                                line_search_fn="strong_wolfe"
                              )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + ("GPU" if torch.cuda.is_available() else "CPU"))
pinn.to(device)


input = torch.tensor(input.astype(np.float32), requires_grad=True).to(device).view(-1, 1)
initial_vals = torch.tensor(initial_vals.astype(np.float32)).to(device).view(-1, 1)
coefficients = torch.tensor(IRK_weights.astype(np.float32)).to(device)

Using device: CPU


Training

In [63]:
i = 0

def closure():
    optimizer.zero_grad() #reset the gradient so that the previous iteration does not affect the current one
    outputs = pinn(input) #run the batch through the current model
    
    overall_loss, SSE_n, SSE_b = SSE(input, outputs, initial_vals, coefficients, pinn, device) #calculate the loss
    overall_loss.backward() #Using backpropagation, calculate the gradients
    #print(f"Avg loss: {loss.item()}")

    global i
    if i%100 == 0:
        print(f"STEP: {i} | Avg Losses | SSE_n: {SSE_n.item()} | SSE_b: {SSE_b.item()} | Total: {overall_loss.item()}")
    i += 1

    return overall_loss

optimizer.step(closure) #Using the gradients, adjust the parameters   

torch.Size([100, 1])
torch.Size([100, 500])
2
torch.Size([100, 500, 100, 500])
torch.Size([100, 500, 100, 1])


AttributeError: 'tuple' object has no attribute 'squeeze'

Evaluation

Graphing

Saving